In [ ]:
include("_.jl")

In [ ]:
so_ge_ = OrderedDict{String,Vector{String}}()

In [ ]:
en = read_ensembl(; or = "mouse")

PR_ = sort(string.(unique((en[en[!, "Gene type"].=="protein_coding", "Gene name"]))))

length(PR_)

## Ensembl

In [ ]:
en = TableAccess.read(joinpath(PAI, "ensembl", "mouse_to_human.tsv.gz"))

en[!, "Gene name"] = string.(en[!, "Gene name"])

;

In [ ]:
mo_hu = Dict{String,String}()

for ro in eachrow(en)

    mo = ro["Gene name"]

    hu = ro["Human gene name"]

    if !(ismissing(mo) || ismissing(hu))

        mo_hu[mo] = hu

    end

end

println(length(mo_hu))

so_ge_["Ensembl"] = [pr for pr in PR_ if !haskey(mo_hu, pr)]

## Write

In [ ]:
hg_ = unique(vcat(values(so_ge_)...))

an_ge_so = DataFrame("Gene" => hg_)

for (so, se_) in so_ge_

    an_ge_so[!, so] = [hg in se_ for hg in hg_]

end

an_ge_so[!, "Sum"] = vec(sum(Matrix(an_ge_so[!, 2:end]); dims = 2))

sort!(an_ge_so, "Sum"; rev = true)

TableAccess.write(joinpath(PAO, "mouse_gene_by_source.tsv"), an_ge_so)

an_ge_so